In [303]:
# Constantes
def OFFICE_ID(i):
    return "office_" + str(i)

def CLIENT_ID(i):
    return "client_" + str(i)

LIMITE_POBLACION_INICIAL = 5000
URL = "./data/fl_3_1"

In [304]:
# Funciones matemáticas
import math

def distance(p1, p2):
    return math.sqrt(math.pow((p1.x - p2.x), 2) + math.pow((p1.y - p2.y), 2))

In [471]:
# Definición de clases
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Client:
    def __init__(self, clientId, point, cost):
        self.point = point
        self.cost = cost
        self.clientId = clientId
        
class Office:
    def __init__(self, officeId, point, cost, storage):
        self.point = point
        self.cost = cost
        self.storage = storage
        self.officeId = officeId
        self.avaibleStorage = storage
    
    def resetStorage(self, storage):
        self.avaibleStorage = storage
    
    def setStorage(self, clientCost):
        self.avaibleStorage = self.avaibleStorage - clientCost
        
class Individual:
    def __init__(self, officesOfClients, idn):
        self.officesOfClients = officesOfClients
        self.idn = idn
    
    def removeGenBySede(self, idn):
        self.officesOfClients =  list(filter(lambda ind: ind[0].officeId != idn, self.officesOfClients))
    
    def addGen(self, gen):
        self.officesOfClients.append(gen)
    
    def addSedeOfClients(self, newAdn):
        self.officesOfClients =  self.officesOfClients + newAdn
        
    def fitness(self):
        size = len(self.officesOfClients)
        cost = 0
        for i in range(0, size):
            sede, cliente = self.officesOfClients[i]
            dist = distance(sede.point, cliente.point)
            cost = cost + (dist * sede.cost)
        return cost
    
    @staticmethod
    def validRestrictions(officesOfClients, offices):
        for office in offices:
            clientsOfOffice = list(filter(lambda officeClient : officeClient[0].officeId == office.officeId, officesOfClients))
            if clientsOfOffice:
                costoTotal = 0
                sede, _ = clientsOfOffice[0]
                almacenamientoSede = sede.storage
                for _ , client in clientsOfOffice:
                    costoTotal = costoTotal + client.cost
                if costoTotal > almacenamientoSede:
                    return False
        return True
    

class Population:
    def __init__(self, individuals):
        self.individuals = individuals
        
    def getIndividualByIndex(self, index):
        return self.individuals[index]
    
    def addIndividual(self, inds):
        self.individuals = self.individuals + inds

In [472]:
# procesamiento de datos
def procesarDatos(url):
    sedes = []
    clientes = []
    with open(url) as f:
        i = 1
        for line in f:
            l = line.split()
            if i == 1:
                cant_sedes = int(l[0])
                cant_clientes = int(l[1])
            else:
                if (cant_sedes > 0):
                    sedes.append(Office(OFFICE_ID(i), Point(float(l[2]), float(l[3])), float(l[0]), float(l[1]) ))
                else:
                    clientes.append(Client(CLIENT_ID(i), Point(float(l[1]), float(l[2])), float(l[0])))
                cant_sedes = cant_sedes - 1    
            i = i + 1
    return (sedes, clientes)

In [473]:
from random import randint

def resetAvaibleStorage(sedes):
    for sede in sedes:
        sede.resetStorage(sede.storage)


def crearPoblacion(numIndividuos, sedes, clientes):
    numSedes = len(sedes)
    numClientes = len(clientes)
    if numSedes <= 10 and numClientes <= 4:
        limit = (numSedes ** numClientes)/3
        if numIndividuos > limit:
             raise Exception('El número de individuos debe ser menor a ' + str(limit))
    if numIndividuos > LIMITE_POBLACION_INICIAL:
        raise Exception('El número de individuo debe ser menor a ' + LIMITE_POBLACION_INICIAL)
                        
    poblacion = []
    j = 0
    while j < numIndividuos:
        individuo = []
        for i in range(0, numClientes):        
            cliente = clientes[i]
            sedesDisponibles = list(filter(lambda sede: sede.avaibleStorage >= cliente.cost, sedes))        
            index =  randint(0, (len(sedesDisponibles) - 1))
            sede = sedesDisponibles[index]
            sede.setStorage(cliente.cost)
            individuo.append((sede, cliente))
        if not individuo in poblacion:
            poblacion.append(Individual(individuo, j))
            j = j + 1
        resetAvaibleStorage(sedes)
    return Population(poblacion)


In [523]:
import random
import math

def resetAvaibleStorage(sedes):
    for sede in sedes:
        sede.resetStorage(sede.storage)

def seleccionPorElitismo(poblacion, sample):
    poblacionTam = len(poblacion.individuals)
    selectInds = int(poblacionTam * sample)
    fit=[]
    for ind in poblacion.individuals:
        fit.append((ind.fitness(), ind))
    return [x[1] for x in sorted(fit, key=lambda student: student[0])[:selectInds]]
    
def seleccionPorTorneo(poblacion, sample, numeroDeParticipantes):
    poblacionTam = len(poblacion.individuals)
    if numeroDeParticipantes > poblacionTam:
        raise Exception('El número de participantes debe ser menor a la población')
    selectInds = int(poblacionTam * sample)
    inds = []
    for i in range(0, selectInds):
        indsTorneo = random.sample(poblacion.individuals, numeroDeParticipantes)
        fit = []
        for ind in indsTorneo:
            fit.append((ind.fitness(), ind))
        inds.append([x[1] for x in sorted(fit, key=lambda student: student[0])[:1]])
    return inds

def mutacion(poblacion, prob, sedes):
    for ind in poblacion.individuals:
        rnd = random.random()     
        if rnd <=  prob:
            x = ind.officesOfClients
            numeroDeGenes = len(x)
            index = random.randint(0, numeroDeGenes-1)
            sedeMutante, _ = x[index]
            sedemMutanteId = sedeMutante.officeId
            clientesMutantes = [cli[1] for cli in list(filter(lambda z: z[0].officeId == sedemMutanteId, x))]
            clientesCost = sum(c.cost for c in clientesMutantes)
            sedeStorage = sedeMutante.storage
            for sede in sedes:
                sedeSustitutaId = sede.officeId
                if sedeSustitutaId != sedemMutanteId:
                    clientesSustitutos = [cli[1] for cli in list(filter(lambda z: \
                                                                      z[0].officeId == sedeSustitutaId, x))]
                    clientesSustitutosCost = sum(c.cost for c in clientesSustitutos)
                    sedeSustitutosStorageStorage = sede.storage
                    if clientesCost <= sedeSustitutosStorageStorage and clientesSustitutosCost <= sedeStorage:
                        ind.removeGenBySede(sedemMutanteId)
                        ind.removeGenBySede(sedeSustitutaId)
                        l1 = [(sede, c) for c in clientesMutantes]
                        l2 = [(sedeMutante, c) for c in clientesSustitutos]
                        ind.addSedeOfClients(l1 + l2)
                        break
    return poblacion  

def getIndexById(sedes, idn):
    return [s for s in sedes if s.officeId == idn][0]
        

def cruce(poblacion, clientes, sedes, hijosSample):
    if hijosSample > 0.4:
        raise Exception('El número de hijos debe ser menor o igual a 0.4')
    numeroDeHijos = math.ceil(hijosSample * len(poblacion.individuals))
    #numeroDePadres = math.ceil(math.sqrt(numeroDeHijos)) * 2
    #samplePadres = numeroDePadres / len(poblacion.individuals)    
    poblacionReproductora = seleccionPorElitismo(poblacion, 1)
    machos = poblacionReproductora[::2]
    hembras = poblacionReproductora[1::2]
    
    print("numeroDeHijos: " + str(numeroDeHijos))
    print("poblacionReproductora: " + str(len(poblacionReproductora)))

    i = 0
    m = 0
    h = 0
    hijos = []
    while(i < numeroDeHijos):
        if len(machos) <= m: 
            break
        padre = machos[m].officesOfClients
        madre = hembras[h].officesOfClients
        hijo = []
        valid = True
        for c in clientes:
            sPadre, cPadre = [(sede, cliente) for (sede, cliente) in padre if cliente.clientId == c.clientId][0]
            sMadre, cMadre = [(sede, cliente) for (sede, cliente) in madre if cliente.clientId == c.clientId][0]
            sedeGenP = [sede for sede in sedes if sede.officeId == sPadre.officeId][0]
            sedeGenM = [sede for sede in sedes if sede.officeId == sMadre.officeId][0]
            sedeValidaP, sedeValidaM = (False, False)
            if sedeGenP.avaibleStorage >= cPadre.cost : sedeValidaP = True
            if sedeGenM.avaibleStorage >= cMadre.cost : sedeValidaM = True
            genHijo = 0
            rnd = random.random()
            if rnd < 0.5 and sedeValidaM: 
                genHijo = (sMadre, cMadre)
                getIndexById(sedes, sMadre.officeId).setStorage(cMadre.cost)
            elif rnd >= 0.5 and sedeValidaP:
                genHijo = (sPadre, cPadre)
                getIndexById(sedes, sPadre.officeId).setStorage(cPadre.cost)
            if genHijo != 0:
                hijo.append(genHijo)
            else:
                valid = False
                break
        resetAvaibleStorage(sedes)
        h = h + 1
        if h == len(hembras):
            m = m + 1
            h = 0
        if valid:
            hijos.append(Individual(hijo, i))
            i = i + 1
    return hijos

In [524]:
#test cruce
sedes, clientes = procesarDatos("./data/fl_3_1")
poblacion = 2
x = crearPoblacion(poblacion, sedes, clientes)
x.addIndividual(cruce(x, clientes, sedes, 0.4))
printPoblacion(x)

numeroDeHijos: 1
poblacionReproductora: 2
----------INDIVIDUO_0-------------------
True
id: office_2
x: 1065.0, y: 1065.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_5
x: 1397.0, y: 1397.0
cost: 50.0

id: office_2
x: 1065.0, y: 1065.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_6
x: 1398.0, y: 1398.0
cost: 50.0

id: office_4
x: 0.0, y: 0.0
cost: 100.0
storage: 500.0
avaibleStorage: 500.0
id: client_7
x: 1399.0, y: 1399.0
cost: 75.0

id: office_3
x: 1062.0, y: 1062.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_8
x: 586.0, y: 586.0
cost: 75.0

----------INDIVIDUO_1-------------------
True
id: office_3
x: 1062.0, y: 1062.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_5
x: 1397.0, y: 1397.0
cost: 50.0

id: office_2
x: 1065.0, y: 1065.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_6
x: 1398.0, y: 1398.0
cost: 50.0

id: office_4
x: 0.0, y: 0.0
cost: 100.0
storage: 500.0
avaibleStorage: 500.0
id: client_7
x: 1399.0,

In [442]:
#test mutacion
sedes, clientes = procesarDatos("./data/fl_3_1")
poblacion = 2
x = crearPoblacion(poblacion, sedes, clientes)
printPoblacion(x)
x = mutacion(x, 1, sedes)
printPoblacion(x)

----------INDIVIDUO_0-------------------
True
id: office_4
x: 0.0, y: 0.0
cost: 100.0
storage: 500.0
avaibleStorage: 500.0
id: client_5
x: 1397.0, y: 1397.0
cost: 50.0

id: office_3
x: 1062.0, y: 1062.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_6
x: 1398.0, y: 1398.0
cost: 50.0

id: office_4
x: 0.0, y: 0.0
cost: 100.0
storage: 500.0
avaibleStorage: 500.0
id: client_7
x: 1399.0, y: 1399.0
cost: 75.0

id: office_4
x: 0.0, y: 0.0
cost: 100.0
storage: 500.0
avaibleStorage: 500.0
id: client_8
x: 586.0, y: 586.0
cost: 75.0

----------INDIVIDUO_1-------------------
True
id: office_4
x: 0.0, y: 0.0
cost: 100.0
storage: 500.0
avaibleStorage: 500.0
id: client_5
x: 1397.0, y: 1397.0
cost: 50.0

id: office_2
x: 1065.0, y: 1065.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_6
x: 1398.0, y: 1398.0
cost: 50.0

id: office_3
x: 1062.0, y: 1062.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_7
x: 1399.0, y: 1399.0
cost: 75.0

id: office_4
x: 0.0, y: 0.0
cos

In [311]:
sedes, clientes = procesarDatos("./data/fl_3_1")
poblacion = 2
x = crearPoblacion(poblacion, sedes, clientes)
printPoblacion(x)
x.individuals[0].removeGenBySede("office_4")
printPoblacion(x)
x.individuals[0].addSedeOfClients([(sedes[0], clientes[0]), (sedes[1], clientes[1])])
printPoblacion(x)


----------INDIVIDUO_0-------------------
True
id: office_3
x: 1062.0, y: 1062.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_5
x: 1397.0, y: 1397.0
cost: 50.0

id: office_4
x: 0.0, y: 0.0
cost: 100.0
storage: 500.0
avaibleStorage: 500.0
id: client_6
x: 1398.0, y: 1398.0
cost: 50.0

id: office_2
x: 1065.0, y: 1065.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_7
x: 1399.0, y: 1399.0
cost: 75.0

id: office_4
x: 0.0, y: 0.0
cost: 100.0
storage: 500.0
avaibleStorage: 500.0
id: client_8
x: 586.0, y: 586.0
cost: 75.0

----------INDIVIDUO_1-------------------
True
id: office_2
x: 1065.0, y: 1065.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_5
x: 1397.0, y: 1397.0
cost: 50.0

id: office_3
x: 1062.0, y: 1062.0
cost: 100.0
storage: 100.0
avaibleStorage: 100.0
id: client_6
x: 1398.0, y: 1398.0
cost: 50.0

id: office_4
x: 0.0, y: 0.0
cost: 100.0
storage: 500.0
avaibleStorage: 500.0
id: client_7
x: 1399.0, y: 1399.0
cost: 75.0

id: office_4
x: 0.0, y: 0

In [213]:
#test seleccionPorTorneo # falta
import random

sedes, clientes = procesarDatos("./data/fl_3_1")
poblacion = 10
x = crearPoblacion(poblacion, sedes, clientes)
seleccionPorTorneo(x, 0.6, 10)

[[<__main__.Individual at 0x7fee1c40c2b0>],

In [214]:
# test selecccionPorElitismo(falta)
sedes, clientes = procesarDatos("./data/fl_3_1")
poblacion = 10
x = crearPoblacion(poblacion, sedes, clientes)
seleccionPorElitismo(x, 0.3)

In [89]:
sedes, clientes = procesarDatos("./data/fl_3_1")
cliente = clientes[0]
sedesDisponibles = list(filter(lambda sede: sede.avaibleStorage >= cliente.cost + 51, sedes))
index =  randint(0, (len(sedesDisponibles) - 1))
sede = sedesDisponibles[index]
sede.setStorage(cliente.cost)

#sedesDisponibles[index].avaibleStorage
print(sedes[2].avaibleStorage)
resetAvaibleStorage(sedes)
print(sedes[2].avaibleStorage)


450.0
500.0


In [499]:
sedes, clientes = procesarDatos("./data/fl_3_1")
poblacion = 7
x = crearPoblacion(poblacion, sedes, clientes)
def printPoblacion(x):
    for ind in x.individuals:
        print("----------INDIVIDUO_" + str(ind.idn) + "-------------------")
        valid = Individual.validRestrictions(ind.officesOfClients, sedes)
        if(valid == False):
            raise Exception('Individuo no valido')
        print(Individual.validRestrictions(ind.officesOfClients, sedes))
        for gen in ind.officesOfClients:
            sede, cliente = gen
            print("id: " + sede.officeId)
            print("x: " + str(sede.point.x) + ", y: " + str(sede.point.y))
            print("cost: " + str(sede.cost))
            print("storage: " + str(sede.storage))
            print("avaibleStorage: " + str(sede.avaibleStorage))
            print("id: " + cliente.clientId)
            print("x: " + str(cliente.point.x) + ", y: " + str(cliente.point.y))
            print("cost: " + str(cliente.cost))
            print()

In [16]:
sedes, clientes = procesarDatos(URL)
print("------------------SEDES----------------------")
for sede in sedes:
    print("id: " + sede.officeId)
    print("x: " + str(sede.point.x) + ", y: " + str(sede.point.y))
    print("cost: " + str(sede.cost))
    print("storage: " + str(sede.storage))
    print()
print("------------------CLIENTES----------------------")
for cliente in clientes:
    print("id: " + cliente.clientId)
    print("x: " + str(cliente.point.x) + ", y: " + str(cliente.point.y))
    print("cost: " + str(cliente.cost))
    print()

------------------SEDES----------------------
id: office_2
x: 1065.0, y: 1065.0
cost: 100.0
storage: 100.0

id: office_3
x: 1062.0, y: 1062.0
cost: 100.0
storage: 100.0

id: office_4
x: 0.0, y: 0.0
cost: 100.0
storage: 500.0

------------------CLIENTES----------------------
id: client_5
x: 1397.0, y: 1397.0
cost: 50.0

id: client_6
x: 1398.0, y: 1398.0
cost: 50.0

id: client_7
x: 1399.0, y: 1399.0
cost: 75.0

id: client_8
x: 586.0, y: 586.0
cost: 75.0



In [17]:
off = Office("1",Point(3.0, 6.0), 233, 122)
off.setStorage(22)
print(off.avaibleStorage)
off.resetStorage(off.storage)
print(off.avaibleStorage)

100
122


In [52]:
sedes, clientes = procesarDatos("./data/fl_3_1")
poblacion = 2
x = crearPoblacion(poblacion, sedes, clientes)
x.individuals[0].officesOfClients[0]

(<__main__.Office at 0x7fee1c3fc5c0>, <__main__.Client at 0x7fee1c3fc550>)

In [355]:
import random

elements = [(1,1,1),(2,3,7),(3,5,10)]
[x[0] for x in elements]
random.random()

0.8968984481299977

In [302]:
l = [1,2,3,4,5]
a = [(i, "a") for i in l]
a

[(1, 'a'), (2, 'a'), (3, 'a'), (4, 'a'), (5, 'a')]

In [354]:
x = [(1, 1), (2, 2), (2, 3), (4, 4)]
[(a, b) for (a, b) in x if a == 3]


[]